# Ingredient embedding


The end goal is wanting to create recipe embeddings that include quantity of each of the ingredients.

To achieve this ingredient names need to be parsed. There are several commercial tools available, vbut I am not willing to pay for an API and there are no other easily usable modules in python. The solution?? Make one!

I have downloaded ingredient data from the NYTimes model which was used to train a model to extract information such as unit, quantity and ingredient name from an ingredient such as "3 yellow squashes, roughly chopped".

I initially thought about this problem as one of a translation problem,  where each term in the ingredient could be labelled as either unit/ingredient etc.
This is actually not that useful as the are complexities within the phrasing that make extracting the actual quantity or the unit from the labelled words difficult. For instance:

"2 tablespoons and 1 teaspoon of sugar" Here there are actually two quantities and units. This labelled dataset does not proide this information so the model would not actually be able to learn to label both. Instead, the label is given as "7 teaspoons".

Instead I will aim to train a combined model with a common ingredient embedding. It is a combined model as there will be two seperate "tails" to the model for the two types of output data:

### Model Input: 

"2 tablespoons and 1 teaspoon of white sugar" -> Remove punctuation
Word Vectorisation -> 

[3, 12, 456, 34, 2304, 304, 78, 6529, 489]

Padding to longest sequence -> 

[3, 12, 456, 34, 2304, 304, 78, 6529, 489, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### Label:

[

    [
        [0 0 0 0 0 0 1 1],      Ingredient Binary Mask
        [0 0 0 0 1 0 0 0]       Unit Binary Mask
    ],
    [
        7.0,                    Quantity Scalar
        0                       Range End (if there is "2 to 4 pears" this would be 4)
    ]
]


### Model:

Embedding Layer (pretrained Glove Embeddings)

LSTM layers

Dense Layers


#### Model Tail 1:
Loss: Binary Crossentropy

Metrics: F1 score for unit and name seperatly

Used as a binary classifier on the input ingredient sentence, where each word is classified for it being an ingredient or not an ingredient. The same is done for the unit and the outputs are stacked:

Given "2 tablespoons and 1 teaspoon of white sugar" it will return [0 0 0 0 0 0 1 1] in the first row to indicate the ingredient and [0 0 0 0 1 0 0 0] in the second dimension to indicate unit.

#### Model Tail 2:
Loss: Huber Loss

Metrics: MSE for the quantity and range end seperatly

Used for a regression on the embedding of the last dense layer, to predict the quantity and the range of the ingredient. 

## Contents:

1. Data Loading and Preprocessing



In [1]:
import pandas as pd

# Load in the labelled ingredient data provided by NYT

ing_df = pd.read_csv("nyt-ingredients-snapshot-2015.csv", index_col="index")

# Drop the columns not needed
ing_df.drop(["comment"], axis=1, inplace=True)
ing_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 179207 entries, 0 to 179206
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   input      179063 non-null  object 
 1   name       178759 non-null  object 
 2   qty        179207 non-null  float64
 3   range_end  173986 non-null  float64
 4   unit       123082 non-null  object 
dtypes: float64(2), object(3)
memory usage: 8.2+ MB


## Preprocessing

A few bits of preprocessing need to happen to the data first

1. Data Cleaning
- Removing punctuation except for . and , as these are common and have contextual meaning
- making . and , seperate words
- Checking that for each row in the dataset the name of the ingredient and the unit are in the input (these are given seperately)
- Removing null inputs (where there is no entry for the input and``  ingredient)
- Dropping rows that dont have a quantity
- replacing nan values in the unit column with 0's

2. Creating labels
- Converting the data to the relevant output (regexp)
- Dropping rows that havent matched to the regexp or dont contain a label

In [2]:
import numpy as np
import re
# Check if the name column has any punctuation - It does have a few rows that have way too much information, I will remove these
def check_if_punct(string: str):
    b = True
    punc_regexp = r"[.,]"
    match = re.search(punc_regexp, string)
    if match:
        b = np.nan
    return b

# Remove rows that dont have the required fields
ing_df.dropna(subset=['input', "name"], inplace=True)
print(f"After removing na values in the name an input, the length of the df is now {len(ing_df)}")

# Convert all the string columns to strings
for col in ['input', 'name', 'unit']:
    ing_df[col] = ing_df[col].apply(str)
    
# Replace all na values in the qty column with 0
ing_df["qty"] = ing_df["qty"].fillna(0)
ing_df["range_end"] = ing_df["range_end"].fillna(0)

ing_df['name_no_punct'] = ing_df['name'].apply(check_if_punct)

# Remove rows that have punctuation in the
ing_df.dropna(subset=["name_no_punct"], inplace=True)
print(f"After removing rows where the name or the input contain commas or full stops, the length of the df is now {len(ing_df)}")

After removing na values in the name an input, the length of the df is now 178668
After removing rows where the name or the input contain commas or full stops, the length of the df is now 174983


The above shows that the filter is working and that rows are being removed. It is also good to see that this filter does not drastically affect the number of rows.

After all the filters have been compelted, then I will manually review subsets of the data to see if there are any further issues or things that have slipped through the net.

In [3]:
import re

# Preprocess the input

def process_punctuation(string):
    string = re.sub(r"([,./])", r" \g<1> ", string)
    string = re.sub(r'[^\w\s.,/]', r" ", string)
    string = re.sub(r"\s+", r" ", string)
    string = re.sub("(\s$)|(^\s)", "", string)

    string = string.lower()
    # Remove trailing white space
    if string[-1] == " ":
        string = string[:-1]
    if string[0] == " ":
        string = string[1:]
    return string


for col in ["input", "name", "unit"]:
    ing_df[col + '_parsed'] = ing_df[col].apply(process_punctuation)


ing_df.head()

,input,name,qty,range_end,unit,name_no_punct,input_parsed,name_parsed,unit_parsed
index,,,,,,,,,
0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,True,1 1 / 4 cups cooked and pureed fresh butternut...,butternut squash,cup
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,True,1 cup peeled and cooked fresh chestnuts about ...,chestnuts,cup
2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,nan,True,"1 medium size onion , peeled and chopped",onion,nan
3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,True,"2 stalks celery , chopped coarse",celery,stalk
4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,True,1 1 / 2 tablespoons vegetable oil,vegetable oil,tablespoon


In the above the string columns are processed to ensure they are all lower case and also to remove any punctuation. The only punctuation I am including is . and , as these are used regularly and provide useful contextual information. finally I am removing multiple whitespace characters.

In [4]:
import re
# Check that the input column contains the ingredient name and also the unit if it has one
def check_if_name_and_unit(input, name, unit):
    name_regexp = f"{name}\w*"
    unit_regexp = f"{unit}\w*"

    match = re.search(name_regexp, input)

    if not match:
        return False
    # Check that the unit isnt nan
    if not unit==unit:
        print(unit)
        match = re.search(unit_regexp, input)
        if not match:
            return False
    return True


ing_df["check_name"] = ing_df.apply(lambda x: check_if_name_and_unit(x['input_parsed'], x['name_parsed'], x['unit_parsed']), axis=1)
ing_df.head()

# Check a few of the entries where it is false
ing_df[~ing_df["check_name"]].head()

,input,name,qty,range_end,unit,name_no_punct,input_parsed,name_parsed,unit_parsed,check_name
index,,,,,,,,,,
253,2 to 3 teaspoons minced jalapeño,jalapeños,2.0,3.0,teaspoon,True,2 to 3 teaspoons minced jalapeño,jalapeños,teaspoon,False
274,Salt and freshly ground black pepper to taste,Salt and black pepper,0.0,0.0,nan,True,salt and freshly ground black pepper to taste,salt and black pepper,nan,False
332,Salt and freshly ground black pepper,Salt and black pepper,0.0,0.0,nan,True,salt and freshly ground black pepper,salt and black pepper,nan,False
347,Salt and freshly ground black pepper,Salt and black pepper,0.0,0.0,nan,True,salt and freshly ground black pepper,salt and black pepper,nan,False
362,Salt and freshly ground pepper,Salt and pepper,0.0,0.0,nan,True,salt and freshly ground pepper,salt and pepper,nan,False


In the above I confirm that the input column has the name and unit in it. This helps to sift out data containing errors.
It also means sifting out data such as input = "2 teaspoons of jalapeños" where the name has been given as name = "jalapeño".
It will remove this data, but I do not believe it will have a significant impact as whether the ingredient is plural or
not it will still be in the same position in the word.

In [5]:
ing_df = ing_df[ing_df["check_name"]]
print(f"After removing all the rows where the name and unit are not included in the input, it leaves {len(ing_df)} rows")



After removing all the rows where the name and unit are not included in the input, it leaves 170302 rows


## Creating labels

For the unit and ingredient labels, I wil need to convert "2 tablespoons and 1 teaspoon of white sugar" into [0 0 0 0 0 0 1 1] for instance.

Now that the data is processed, this should not be too difficult.

In [6]:
# Convert a string of multiple words into a binary array with 1 for each word in 
# Match is and 0 where any other words are.
def create_label(inp, match):
    match_regexp = f"{match}\w*"
    match_num_words = len(match.split(" "))                  
    match_replaced = re.sub(match_regexp, " MATCH ", inp)
    match_replaced = re.sub("\s+", " " ,match_replaced)
    match_replaced = re.sub("(\s$)|(^\s)", "", match_replaced)
    word_array = match_replaced.split(" ")
    label = []
    for word in word_array:
        if word == "MATCH":
            label.extend([1]*match_num_words)
        else:
            label.append(0)
    return label
        
ing_df["name_label"] = ing_df.apply(lambda x: create_label(x['input_parsed'], x['name_parsed']), axis=1)
ing_df["unit_label"] = ing_df.apply(lambda x: create_label(x['input_parsed'], x['unit_parsed']), axis=1)



In [7]:


ing_df_labelled = ing_df[["input_parsed", 'name_label', "unit_label", "qty", "range_end"]]
ing_df_labelled.sample(100)

,input_parsed,name_label,unit_label,qty,range_end
index,,,,,
14592,1 tablespoon fresh lemon juice,"[0, 0, 1, 1, 1]","[0, 1, 0, 0, 0]",1.00,0.0
81373,pinch salt,"[0, 1]","[1, 0]",1.00,0.0
125617,"2 garlic cloves , minced","[0, 1, 0, 0, 0]","[0, 0, 0, 0, 0]",2.00,0.0
78967,"4 fillets of black sea bass , skins on about 1...","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",4.00,0.0
174044,"2 medium garlic cloves , thinly sliced","[0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0]",2.00,0.0
...,...,...,...,...,...
63679,1 / 4 teaspoon grated lemon rind,"[0, 0, 0, 0, 0, 1, 1]","[0, 0, 0, 1, 0, 0, 0]",0.25,0.0
99317,8 peppercorns,"[0, 1]","[0, 0]",8.00,0.0
16585,1 teaspoon sugar,"[0, 0, 1]","[0, 1, 0]",1.00,0.0


These look pretty good. The next stage would be to encode the text so that it is also numeric.

To do this I need to:
- Identify the vocabulary from the corpus
- Convert the words into their numeric representation
- Pad the sequences so that all the inputs are the same length
- Download the pretrained word embeddings (using Glove 6)


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from more_itertools import take

# Initialize the Tokenizer class, making sure to keep , . and /
tokenizer = Tokenizer(oov_token="<OOV>", filters='!"#$%&()*+-:;<=>?@[\\]^_`{|}~\t\n')

# Generate the word index dictionary
tokenizer.fit_on_texts(ing_df_labelled["input_parsed"])

# Print the length of the word index
word_index_example = take(20, tokenizer.word_index.items())
word_index = tokenizer.word_index
print(f'number of words in word_index: {len(word_index)}')

# Print the word index
print(f'word_index: {word_index_example}')
print()

# Generate and pad the sequences
sequences = tokenizer.texts_to_sequences(ing_df_labelled["input_parsed"])

X = pad_sequences(sequences, padding='post')
y_name = pad_sequences(ing_df_labelled["name_label"], padding="post")
y_unit = pad_sequences(ing_df_labelled["unit_label"], padding="post")
y_qty = np.expand_dims(np.asarray(ing_df_labelled["qty"].values), axis=-1)
y_range = np.expand_dims(np.asarray(ing_df_labelled["range_end"].values), axis=-1)

y_qtys = np.concatenate((y_qty, y_range), axis=-1)

# Combine these for jointly training the model
y_name_ex = np.expand_dims(y_name, axis=1)
y_unit_ex = np.expand_dims(y_unit, axis=1)
print(y_qtys.shape)

y_combined = np.concatenate((y_name_ex, y_unit_ex), axis=1)

number of words in word_index: 7501
word_index: [('<OOV>', 1), ('1', 2), ('2', 3), (',', 4), ('/', 5), ('cup', 6), ('4', 7), ('tablespoons', 8), ('3', 9), ('teaspoon', 10), ('or', 11), ('chopped', 12), ('and', 13), ('to', 14), ('salt', 15), ('cups', 16), ('oil', 17), ('tablespoon', 18), ('fresh', 19), ('pepper', 20)]

(170302, 2)


In [9]:
import random 
# Check a random index to make sure everything has worked
idx = random.randint(0,20000)
rand_ing = ing_df.loc[ing_df.index.values[idx], "input_parsed"]
print(f"Ingredient sentence is {rand_ing}")
print(f"X is {X[idx]}")
print(f"y_name is {y_name[idx]}")
print(f"y_unit is {y_unit[idx]}")

Ingredient sentence is 2 tablespoons rum
X is [  3   8 342   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0]
y_name is [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
y_unit is [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


Great so we have a training database! 

## Word Embeddings

To save time training my own embeddings, I will use the ones already trained in the Glove version 6 embeddings. These were created by stanford on a much larger corpus. Naturally some words might not exsist in this corpus, so these will be replaced with the OOV token.

Below I download and save the dataset locally

In [10]:
# Download and extract the word embeddings
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip


In [11]:
# Open the word embeddings 
import os
import numpy as np


path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [12]:
num_tokens = len(word_index) + 1
embedding_dim = 100
hits = 0
misses = 0
oov_words = []

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        oov_words.append(word)
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
print(f"Some examples of the words not included are: {oov_words[:20]}")

Converted 5915 words (1586 misses)
Some examples of the words not included are: ['<OOV>', 'href', 'fraîche', 'puréed', 'deribbed', 'calamata', '1½', 'pimentón', 'cherrystone', 'zested', 'dukkah', 'debearded', 'frisée', 'niçoise', 'medjool', 'comice', 'florettes', 'unblanched', 'borlotti', 'porcinis']


In [13]:
# Remove these words from the word index

If I looked at a longer list I would see a lot of large numeric values. It appears that larger numbers do not have embeddings. To improve the model in the future, all numbers could be given a numeric token and all words containing non-english characters could be replaced for their english equivilant. THen any remaining words would be replaced with the OOV token. Then the embedding layer could be partially fixed and train embeddings for the new tokens.

Because of the current approach, OOV words are replaced with a 0 so they are treated the same as the padding. This is unlikely to have a significant effect, except for when the ingredient name is at the end of the phrase. Lets see how it does anyway.

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
# Create the embedding layer, make it not trainable and fix the embedding values
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
    input_length=X.shape[1]
)

## Baseline Calculation

If my model was to output all 1's or all 0's for the output, what would the success rate be?

In [15]:
from sklearn.metrics import accuracy_score, f1_score

baseline_y_unit = f1_score(y_unit, np.ones(y_unit.shape), average="micro")
baseline_y_name = f1_score(y_name, np.ones(y_name.shape), average="micro")

print(f"The baseline f1 scores for y_unit are {baseline_y_unit}")
print(f"The baseline f1 scores for y_name are {baseline_y_name}")

The baseline f1 scores for y_unit are 0.02337087910892176
The baseline f1 scores for y_name are 0.055756400440218165


## Metrics 

In [16]:
import tensorflow_addons as tfa
import tensorflow as tf

f1_score_name = tfa.metrics.F1Score(
    num_classes = X.shape[1],
    average = "micro",
    threshold=0.5
)

f1_score_unit = tfa.metrics.F1Score(
    num_classes = X.shape[1],
    average = "micro",
    threshold=0.5
)


def name_f1_score(y_true_comb, y_pred_comb):
    y_true_name, y_true_unit = tf.split(y_true_comb, num_or_size_splits=[1, 1], axis=1)
    y_pred_name, y_pred_unit = tf.split(y_pred_comb, num_or_size_splits=[1, 1], axis=1)
    return f1_score_name(tf.squeeze(y_true_name), tf.squeeze(y_pred_name))

def unit_f1_score(y_true_comb, y_pred_comb):
    y_true_name, y_true_unit = tf.split(y_true_comb, num_or_size_splits=[1, 1], axis=1)
    y_pred_name, y_pred_unit = tf.split(y_pred_comb, num_or_size_splits=[1, 1], axis=1)
    return f1_score_unit(tf.squeeze(y_true_unit), tf.squeeze(y_pred_unit))


def name_percent_part_correct(y_true_comb, y_pred_comb):
    y_true_name, y_true_unit = tf.split(y_true_comb, num_or_size_splits=[1, 1], axis=1)
    y_pred_name, y_pred_unit = tf.split(y_pred_comb, num_or_size_splits=[1, 1], axis=1)
    
    metric = percent_part_correct(tf.squeeze(y_true_name), tf.squeeze(y_pred_name))
    return metric

def percent_part_correct(y_true, y_pred):
    y_round = tf.cast(tf.math.round(y_pred), tf.bool)
    y_true_bool = tf.cast(y_true, tf.bool)
    total_values = tf.gather(tf.shape(y_true), 0)
    
    # Returns a boolean tensor where the elements match
    equal = tf.math.logical_and(y_true_bool, y_round)
    only_ones = tf.logical_and(equal, y_true_bool)
    reduced_eq = tf.reduce_any(only_ones, 1)
    total_part_correct = tf.reduce_sum(tf.cast(reduced_eq, tf.int64))
    percent_part_correct = tf.math.divide_no_nan(tf.cast(total_part_correct, tf.float32), tf.cast(total_values, tf.float32))
    return percent_part_correct

def qty_mse(y_true, y_pred):
    y_true_qty, y_true_range = tf.split(y_true, num_or_size_splits=[1, 1], axis=-1)
    y_pred_qty, y_pred_range = tf.split(y_pred, num_or_size_splits=[1, 1], axis=-1)
    mse = tf.keras.losses.MeanSquaredError()
    mse_error = mse(y_true_qty, y_pred_qty)
    return mse_error

def range_mse(y_true, y_pred):
    y_true_qty, y_true_range = tf.split(y_true, num_or_size_splits=[1, 1], axis=-1)
    y_pred_qty, y_pred_range = tf.split(y_pred, num_or_size_splits=[1, 1], axis=-1)
    mse = tf.keras.losses.MeanSquaredError()
    mse_error = mse(y_true_range, y_pred_range)
    return mse_error


# Test execution of custom metric
y_p = tf.constant([[[0, 0.51, 0], [0.51, 0, 0]], [[1, 0, 0], [0, 0, 0]]], dtype=tf.float32)
y_t = tf.constant([[[0, 1, 0], [1, 0, 0]], [[0, 0, 0], [0, 0, 0]]], dtype=tf.float32)

print(name_percent_part_correct(y_combined, y_combined).numpy())

# Test the custom f1_score
print(unit_f1_score(y_t, y_p))

2022-03-21 07:35:56.335429: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1.0
tf.Tensor(1.0, shape=(), dtype=float32)


## Split Data

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_combined, test_size=0.2, random_state=42)
_, _, y_qty_train, y_qty_test = train_test_split(X, y_qty, test_size=0.2, random_state=42)
_, _, y_range_train, y_range_test = train_test_split(X, y_range, test_size=0.2, random_state=42)

## Build and Train Model

In [35]:


lstm_dim = 64
dense_dim = 128
output_dim = X.shape[1]*2
NUM_EPOCHS = 50
BATCH_SIZE = 64

# Model Definition with LSTM
model_head = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim, return_sequences=True)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(dense_dim, activation='relu')
], name="model_head")
# Print the model summary
model_head.summary()

save_freq = int(5.0*(len(X_train)/BATCH_SIZE))
print(save_freq)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('models/model{epoch:08d}.ckpt', save_freq=save_freq, save_weights_only="true")
                                                                        



Model: "model_head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           750200    
                                                                 
 bidirectional_14 (Bidirecti  (None, 60, 128)          84480     
 onal)                                                           
                                                                 
 bidirectional_15 (Bidirecti  (None, 60, 128)          98816     
 onal)                                                           
                                                                 
 flatten_7 (Flatten)         (None, 7680)              0         
                                                                 
 dense_16 (Dense)            (None, 128)               983168    
                                                                 
Total params: 1,916,664
Trainable params: 1,166,464
Non-

In [36]:
model_tail_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(output_dim, activation='sigmoid'), 
    tf.keras.layers.Reshape((2, X.shape[1]))
], name="model_tail_1")

model_tail_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='relu')
], name="model_tail_2")

model_tail_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='relu')
], name="model_tail_3")

model_input = tf.keras.layers.Input(shape=[X.shape[1]])
features = model_head(model_input)
model_output1 = model_tail_1(features)
model_output2 = model_tail_2(features)
model_output3 = model_tail_3(features)

model = tf.keras.Model(inputs=model_input, outputs=[model_output1, model_output2, model_output3])

TAIL1_WEIGHT = 4
TAIL2_WEIGHT = 1
# Weight is the ratio of positive samples so that the loss is of the same magnitude as the QTY loss
TAIL3_WEIGHT = float(len(y_range_train)) / sum([1.0 for i in y_range_train if i > 0.01]) 

model.compile(
    optimizer="Adam",
    loss={"model_tail_1": "binary_crossentropy", "model_tail_2": "huber", "model_tail_3": "mse"},
    loss_weights={"model_tail_1": TAIL1_WEIGHT, "model_tail_2": TAIL2_WEIGHT, "model_tail_3": TAIL3_WEIGHT},
    metrics={"model_tail_1": [name_f1_score, unit_f1_score, name_percent_part_correct], "model_tail_2": "mse", "model_tail_3": "mse"}
)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 model_head (Sequential)        (None, 128)          1916664     ['input_4[0][0]']                
                                                                                                  
 model_tail_1 (Sequential)      (None, 2, 60)        15480       ['model_head[0][0]']             
                                                                                                  
 model_tail_2 (Sequential)      (None, 1)            129         ['model_head[0][0]']             
                                                                                            

In [37]:
print(TAIL3_WEIGHT)

83.58343558282209


In [38]:
model.fit(X_train, [y_train, y_qty_train, y_range_train], batch_size=BATCH_SIZE, 
                              epochs=NUM_EPOCHS, validation_data=(X_test, [y_test, y_qty_test, y_range_test]), callbacks= [checkpoint_callback])

Epoch 1/50
2129/2129 [==============================] - 223s 102ms/step - loss: 80.3023 - model_tail_1_loss: 0.0315 - model_tail_2_loss: 0.6573 - model_tail_3_loss: 0.9514 - model_tail_1_name_f1_score: 0.8692 - model_tail_1_unit_f1_score: 0.9067 - model_tail_1_name_percent_part_correct: 0.7349 - model_tail_2_mse: 176.3011 - model_tail_3_mse: 0.9514 - val_loss: 54.9397 - val_model_tail_1_loss: 0.0157 - val_model_tail_2_loss: 0.5300 - val_model_tail_3_loss: 0.6502 - val_model_tail_1_name_f1_score: 0.8678 - val_model_tail_1_unit_f1_score: 0.9060 - val_model_tail_1_name_percent_part_correct: 0.9461 - val_model_tail_2_mse: 226.6915 - val_model_tail_3_mse: 0.6502
Epoch 2/50
2129/2129 [==============================] - 213s 100ms/step - loss: 80.0294 - model_tail_1_loss: 0.0134 - model_tail_2_loss: 0.4562 - model_tail_3_loss: 0.9514 - model_tail_1_name_f1_score: 0.8679 - model_tail_1_unit_f1_score: 0.9061 - model_tail_1_name_percent_part_correct: 0.9713 - model_tail_2_mse: 125.9799 - model_ta

NotFoundError: models; No such file or directory

## Save Model

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model('./final_model.pb', custom_objects= {"embedding": embedding_layer, "name_f1_score": name_f1_score, "unit_f1_score": unit_f1_score, "name_percent_part_correct": name_percent_part_correct, "qty_mse": qty_mse, "range_mse": range_mse})

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('output_model.tflite', 'wb') as f:
    f.write(tflite_model)

## Visualize Training

In [ ]:
import plotly.graph_objects as go

def plot_graphs(history, strings):
    fig = go.Figure()
    x = list(range(NUM_EPOCHS))
    for string in strings:
        fig.add_scatter(x=x, y=history.history[string],
                        name=string)
        fig.add_scatter(x=x, y=history.history['val_'+string],
                        name='val_'+string)
    fig.update_traces(mode='lines+markers')# hoverinfo='text+name+y', 
    fig.update_layout(legend=dict(y=0.5, traceorder='reversed', font_size=16), 
                    title="Training Results")
    fig.show()

plot_graphs(model.history, ["model_tail_1_name_f1_score", "model_tail_1_unit_f1_score"])
plot_graphs(model.history, ["loss", "model_tail_1_loss", "model_tail_2_loss", "model_tail_3_loss"])
plot_graphs(model.history, ['model_tail_1_name_percent_part_correct'])
plot_graphs(model.history, ['model_tail_2_qty_mse', 'model_tail_2_range_mse'])


The above loss graph can be used to adjust the weights on each of the contributing losses based on when each tail starts to overfit, ideally they will all start to overfit at the same time.

## Analyse Success and Failure Cases

In [ ]:
def test_combined_model(n_samples, qty_failure=False, has_range=False):
    start_index = 100000
    subset_size = 10000
    X_sentences = ing_df_labelled.loc[ing_df_labelled.index[start_index:start_index+subset_size], "input_parsed"].values
    X_input = X[start_index:start_index+subset_size]

    y_name_subset = y_name[start_index:start_index+subset_size]
    y_unit_subset = y_unit[start_index:start_index+subset_size]
    y_qty_subset  = y_qty[start_index:start_index+subset_size]
    y_range_subset = y_range[start_index:start_index+subset_size]
    pred_y_subset = model.predict(X[start_index:start_index+subset_size])
    

    pred_y_name = pred_y_subset[0][:, 0, :]
    pred_y_unit = pred_y_subset[0][:, 1, :]
    pred_y_qty  = pred_y_subset[1]
    pred_y_range = pred_y_subset[2]
        
    rounded_yp_name = np.rint(np.asarray(pred_y_name))
    rounded_yp_unit = np.rint(np.asarray(pred_y_unit))

    for i in range(n_samples):
        if qty_failure:
            if np.abs(pred_y_qty[i] - y_qty_subset[i]) > 3:
                pass
            else:
                continue
        if has_range:
            if y_range_subset[i] > 0.01:
                pass
            else:
                continue
            
        print(f"Sentence: {X_sentences[i]}")
        print()
        print(f"y_name: {binary_mask_to_words(y_name_subset[i], X_sentences[i])}")
        print(f"pred_name: {binary_mask_to_words(rounded_yp_name[i], X_sentences[i])}")
        print()
        print(f"y_unit: {binary_mask_to_words(y_unit_subset[i], X_sentences[i])}")
        print(f"pred_unit: {binary_mask_to_words(rounded_yp_unit[i], X_sentences[i])}")
        print()
        print(f"y_qty: {y_qty_subset[i]}")
        print(f"pred_qty: {pred_y_qty[i]}")
        print()
        print(f"y_range: {y_range_subset[i]}")
        print(f"pred_range: {pred_y_range[i]}")
        print()

# Applies a binary mask to a sentence and returns the strings. Used for calculating the 
def binary_mask_to_words(binary_mask, sentence):
    split_sentence = sentence.split(" ")
    idxs = np.where(binary_mask > 0.5)
    y_words = " ".join([split_sentence[int(idx)] for idx in idxs[0]])
    return y_words
    
test_combined_model(10000, has_range=True)



One interesting failure (or maybe a success?):

3 / 4 cup plus 2 tablespoons sugar -> Model: 9.98 Tablespoons sugar Label: 3/4 cup sugar

3/4 cups is actually 10 tablespoons, so the model answer is just as good as the true answer. If only it were able to add the additional 2 teaspoons as well!



## Identify fail cases

Because the data I have used has had the labels converted and because it is text data (I find it complicated and havent used it before), I want to see where the model has failed. This will helo to refine the preprocesing and dta filtering. It might be errors from the differences between what different labelers consider to be the ingredient.

In [ ]:
def print_examples(fail_pass: str, part_correct: bool, name_unit: str, limit: int):
    start_index = 120000
    subset_size = 1000
    X_sentences = ing_df_labelled.loc[ing_df_labelled.index[start_index:start_index+subset_size], "input_parsed"].values
    X_input = X[start_index:start_index+subset_size]
    if name_unit == "name":
        y_subset = y_name[start_index:start_index+subset_size]
    else:
        y_subset = y_unit[start_index:start_index+subset_size]
        
    pred_y_subset = model_lstm.predict(X[start_index:start_index+subset_size])
    
    if name_unit == "name":
        pred_y_subset = pred_y_subset[:, 0, :]
    else:
        pred_y_subset = pred_y_subset[:, 1, :]
        
    rounded_y_pred = np.rint(np.asarray(pred_y_subset))
    
    printed = 0
    # Print any errors
    for i in range(subset_size):
        if printed > limit - 1:
            return
        if fail_pass == "fail":
            if part_correct:
                # Checks that there is at least one matching positive and then checks that there is not a perfect match.
                if (np.asarray(y_subset[i]) & rounded_y_pred[i].astype(int)).any() and not (np.asarray(y_subset[i]) == rounded_y_pred[i]).all():
                    print(f"Sentence: {X_sentences[i]}")
                    print(f"y: {binary_mask_to_words(y_subset[i], X_sentences[i])}")
                    print(f"pred_y: {binary_mask_to_words(rounded_y_pred[i], X_sentences[i])}")
                    print()
                    printed += 1
            else:
                # Checks that there is at least one word that is a name or unit
                # Then finds out where the overlap is between the y_true and the y_pred, bthen combines it with the y_pred array using and 
                # to remove the matches where both are 0
                # Then checks that there are no 1's in this array, meaning a compelte error
                if np.any(np.asarray(y_subset[i])):
                    mask = (np.asarray(y_subset[i]) == rounded_y_pred[i]) & np.asarray(y_subset[i])
                    if not mask.any():
                        print(f"Sentence: {X_sentences[i]}")
                        print(f"y: {binary_mask_to_words(y_subset[i], X_sentences[i])}")
                        print(f"pred_y: {binary_mask_to_words(rounded_y_pred[i], X_sentences[i])}")
                        print()
                        printed += 1

        if fail_pass == "pass":
            if (np.asarray(y_subset[i]) == rounded_y_pred[i]).all():
                print(f"Sentence: {X_sentences[i]}")
                print()
                printed += 1




## Failed Name cases

In [ ]:
print_examples("fail", True, "name", 3)

It appears that there are two kind of errors. Errors where the model has clearly gotten it wrong:

1 pinch sea salt or fleur de sel -> Model Output: "Sea Salt Or"

And others which are open to interpretation or where the labeller has gotten it wrong:

1 red grapefruit , peeled and segmented , optional -> Model Output: "red grapefruit" Label: "grapefruit"

1 / 4 cup vadouvan exotique spice mix -> Model Output: "vadouvan exotique spice mix" Label: "spice mix"

2 teaspoons aged balsamic vinegar -> Model Output: "balsamic vinegar" Label: "vinegar"

I cannot think of any straightforward ways to seperate these mistakes, however, I can create a metric that includes the above examples as successful predictions and then have a look at the ones that are not these type of errors. This would be useful as a better predictor of model performance I believe.

This metric would include any prediction with an overlap with the Label a successful prediction and then give a percentage of the data set that fall into this category.

In [ ]:
print_examples("fail", False, "name", 3)

From the aboe example of errors, it looks like the algorithm is actually better than the human labeller in this instance.

crackers or sliced cucumber , for serving -> Model: crackers, Label: cucumber
1 / 2 cup soaking water from the apricots , as needed - > Model: water Label: apricots

## Failed Unit Cases

In [ ]:
print("Successful Predictions")
print_examples("pass", False, "unit", 10)

In [ ]:
print("Unsuccessful Predictions")
print_examples("fail", False, "unit", 5)

In [ ]:
print("Unsuccessful predictions where part of the prediction is correct")
print_examples("fail", True, "unit", 100)